In [21]:
#modules
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests

### Teams Data
This will ultimately be the main focus dataset. We want to review team seasonal averages, but we have player data to see how positions affect different team metrics. We extract this for the last ten seasons (2010-2019) from the official [NBA website](https://stats.nba.com/teams/traditional/?Season=2019-20&SeasonType=Regular%20Season). Unlike the other data from Sports Reference, it was easier to scrape the NBA website because the SR data for season average is stored as HTML comments. It is a fun extraction challenge, but for another day.

In [44]:
seas = '2011-12'
team_URL = 'https://stats.nba.com/teams/traditional/?Season=' +seas+ '&SeasonType=Regular%20Season'
#webdriver and go to site and get dynamic table
dr = webdriver.Chrome(executable_path=r'C:/bin/chromedriver.exe')
dr.get(team_URL)
table = dr.find_element_by_xpath('//div[@class="nba-stat-table"]//div[1]//table').get_attribute('outerHTML')
#read table
raw_table = pd.read_html(table)[0]
raw_table.head()

,Unnamed: 0,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB RANK,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK
0,1,Milwaukee Bucks,82,60,22,0.732,48.2,118.1,43.4,91.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Toronto Raptors,82,58,24,0.707,48.5,114.4,42.2,89.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Golden State Warriors,82,57,25,0.695,48.3,117.7,44.0,89.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Denver Nuggets,82,54,28,0.659,48.1,110.7,41.9,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Houston Rockets,82,53,29,0.646,48.4,113.9,39.2,87.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#make a copy before cleaning, raw_table does not appear if I try to use it
raw_data = raw_table.copy()

,Unnamed: 0,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB RANK,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,BLKA RANK,PF RANK,PFD RANK,+/- RANK
0,1,Milwaukee Bucks,82,60,22,0.732,48.2,118.1,43.4,91.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Toronto Raptors,82,58,24,0.707,48.5,114.4,42.2,89.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Golden State Warriors,82,57,25,0.695,48.3,117.7,44.0,89.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Denver Nuggets,82,54,28,0.659,48.1,110.7,41.9,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Houston Rockets,82,53,29,0.646,48.4,113.9,39.2,87.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Portland Trail Blazers,82,53,29,0.646,48.4,114.7,42.3,90.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Philadelphia 76ers,82,51,31,0.622,48.3,115.2,41.5,88.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Utah Jazz,82,50,32,0.610,48.2,111.7,40.4,86.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Boston Celtics,82,49,33,0.598,48.2,112.4,42.1,90.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Oklahoma City Thunder,82,49,33,0.598,48.4,114.5,42.6,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We only need the data columns because the rank columns are dynamic and if we ever need to determine this, we would be able to do so by sorting specific columns

In [76]:
raw_data.columns
data = raw_data[['TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM',
       'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB',
       'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', '+/-']]
data.head()

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Milwaukee Bucks,82,60,22,0.732,48.2,118.1,43.4,91.1,47.6,...,40.4,49.7,26.0,13.9,7.5,5.9,4.8,19.6,20.2,8.9
1,Toronto Raptors,82,58,24,0.707,48.5,114.4,42.2,89.1,47.4,...,35.6,45.2,25.4,14.0,8.3,5.3,4.5,21.0,20.5,6.1
2,Golden State Warriors,82,57,25,0.695,48.3,117.7,44.0,89.8,49.1,...,36.5,46.2,29.4,14.3,7.6,6.4,3.6,21.4,19.5,6.5
3,Denver Nuggets,82,54,28,0.659,48.1,110.7,41.9,90.0,46.6,...,34.5,46.4,27.4,13.4,7.7,4.4,5.0,20.0,20.4,4.0
4,Houston Rockets,82,53,29,0.646,48.4,113.9,39.2,87.4,44.9,...,31.9,42.1,21.2,13.3,8.5,4.9,4.5,22.0,20.0,4.8


Putting it all together, we need to:
- Extract information
- Only include columns of interest
- Add in the year for which the season applies
- Append values into a single dataframe
- Add in end of season champion and runner-up indicators

In [138]:
def get_data(season, URL):
    #access web driver and access page content
    dr = webdriver.Chrome(executable_path=r'C:/bin/chromedriver.exe')
    dr.get(URL)
    table = dr.find_element_by_xpath('//div[@class="nba-stat-table"]//div[1]//table').get_attribute('outerHTML')
    #get the data metrics for the team into a dataframe
    raw_df = pd.read_html(table)[0]
    data = raw_df[['TEAM', 'GP', 'W', 'L', 'WIN%', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 
        'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', '+/-']]
    data['Year'] = int(i)
    dr.close()
    return data

In [139]:
df_team = pd.DataFrame()
#get each year's data
for i in range(2010,2020):
    seas = str(i-1)+'-'+str(i)[2:]
    teams_URL = 'https://stats.nba.com/teams/traditional/?Season=' +seas+ '&SeasonType=Regular%20Season'
    df_team = pd.concat([df_team, get_data(seas, teams_URL)], ignore_index=True)

In [140]:
df_team.head()

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,Year
0,Cleveland Cavaliers,82,61,21,0.744,48.2,102.1,37.8,77.9,48.5,...,42.5,22.4,13.9,6.9,5.2,4.0,19.4,20.8,6.5,2010
1,Orlando Magic,82,59,23,0.720,48.1,102.8,36.6,78.0,47.0,...,43.2,19.7,14.1,6.2,5.6,3.5,19.9,22.2,7.5,2010
2,Los Angeles Lakers,82,57,25,0.695,48.4,101.7,38.3,83.8,45.7,...,44.3,21.1,13.4,7.5,4.9,4.4,19.4,21.2,4.7,2010
3,Dallas Mavericks,82,55,27,0.671,48.5,102.0,38.3,82.4,46.4,...,41.7,23.4,12.9,7.6,5.5,4.1,19.1,20.1,2.7,2010
4,Phoenix Suns,82,54,28,0.659,48.1,110.2,40.7,82.8,49.2,...,43.0,23.3,14.8,5.8,5.1,4.5,20.9,21.8,4.9,2010


In [144]:
#standardize names
df_team[df_team['TEAM']=='LA Clippers']['TEAM'] = 'Los Angeles Clippers'

In [145]:
df_team.dtypes

TEAM     object
GP        int64
W         int64
L         int64
WIN%    float64
MIN     float64
PTS     float64
FGM     float64
FGA     float64
FG%     float64
3PM     float64
3PA     float64
3P%     float64
FTM     float64
FTA     float64
FT%     float64
OREB    float64
DREB    float64
REB     float64
AST     float64
TOV     float64
STL     float64
BLK     float64
BLKA    float64
PF      float64
PFD     float64
+/-     float64
Year      int64
dtype: object

Adding champion and runner-up indicators for the seasons. Also note again that 2011-12 was a lockout shortened season. Note that the champions file has abbreviations and team names, while this data has only fully spelled out names. We need to merge the championship status using these columns.

In [146]:
#get champions data
champions = pd.read_csv('champions.csv', index_col='Unnamed: 0')
champions.head()

,Year,Champion,Runner-Up,Finals MVP,Top Scorer,Top Rebr,Top Asst,WS Lead,Points,Rebounds,Assists,Win Shares,Champ-Abbr,Runner-Up-Abbr
0,2019,Toronto Raptors,Golden State Warriors,K. Leonard,K. Leonard (732),D. Green (223),D. Green (187),K. Leonard (4.9),732,223,187,4.9,TOR,GSW
1,2018,Golden State Warriors,Cleveland Cavaliers,K. Durant,L. James (748),D. Green (222),L. James (198),L. James (5.2),748,222,198,5.2,GSW,CLE
2,2017,Golden State Warriors,Cleveland Cavaliers,K. Durant,L. James (591),K. Love (191),L. James (141),L. James (4.3),591,191,141,4.3,GSW,CLE
3,2016,Cleveland Cavaliers,Golden State Warriors,L. James,K. Thompson (582),D. Green (228),R. Westbrook (198),L. James (4.7),582,228,198,4.7,CLE,GSW
4,2015,Golden State Warriors,Cleveland Cavaliers,A. Iguodala,L. James (601),D. Howard (238),L. James (169),S. Curry (3.9),601,238,169,3.9,GSW,CLE


In [174]:
#new columns to check merge for
df_team['Champ'] = 0
df_team['Runner-Up'] = 0

#check team status
for champ, ru, year in zip(champions.Champion.values, champions['Runner-Up'].values, champions.Year.values):
    df_team['Champ'][(df_team['TEAM'] == champ) & (df_team['Year']== year)] = 1
    df_team['Runner-Up'][(df_team['TEAM']== ru) & (df_team['Year']== year)] = 1

In [178]:
#update team so that it is abbreviations and not long names
#we only want the most recent abbreviations so exclude the bottom six which were replaced 
team_names = pd.read_csv('team_names.csv', index_col='Unnamed: 0')[:-6]
team_names = team_names.rename(columns={'Franchise':'TEAM'}) #update name to be able to merge
df_team = pd.merge(df_team, team_names, on='TEAM')
df_team = df_team.rename(columns={'Key':'Tm'})

In [188]:
#checking merge happened correctly by looking at the last few years
check = df_team[(df_team['Champ']==1) | (df_team['Runner-Up']==1)]
check.sort_values(by=['Year', 'Runner-Up'])[10:]

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,STL,BLK,BLKA,PF,PFD,+/-,Year,Champ,Runner-Up,Tm
301,Golden State Warriors,82,67,15,0.817,48.1,110.0,41.6,87.0,47.8,...,9.3,6.0,3.6,19.9,18.8,10.1,2015,1,0,GSW
5,Cleveland Cavaliers,82,53,29,0.646,48.2,103.1,37.7,82.2,45.8,...,7.4,4.1,4.5,18.4,20.5,4.5,2015,0,1,CLE
6,Cleveland Cavaliers,82,57,25,0.695,48.4,104.3,38.7,84.0,46.0,...,6.7,3.9,4.4,20.3,20.6,6.0,2016,1,0,CLE
302,Golden State Warriors,82,73,9,0.890,48.5,114.9,42.5,87.3,48.7,...,8.4,6.1,4.1,20.7,19.8,10.8,2016,0,1,GSW
303,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,...,9.6,6.8,3.8,19.3,19.4,11.6,2017,1,0,GSW
7,Cleveland Cavaliers,82,51,31,0.622,48.5,110.3,39.9,84.9,47.0,...,6.6,4.0,4.3,18.1,20.6,3.2,2017,0,1,CLE
304,Golden State Warriors,82,58,24,0.707,48.1,113.5,42.8,85.1,50.3,...,8.0,7.5,3.7,19.6,18.5,6.0,2018,1,0,GSW
8,Cleveland Cavaliers,82,50,32,0.610,48.1,110.9,40.4,84.8,47.6,...,7.1,3.8,4.1,18.6,20.7,0.9,2018,0,1,CLE
249,Toronto Raptors,82,58,24,0.707,48.5,114.4,42.2,89.1,47.4,...,8.3,5.3,4.5,21.0,20.5,6.1,2019,1,0,TOR
305,Golden State Warriors,82,57,25,0.695,48.3,117.7,44.0,89.8,49.1,...,7.6,6.4,3.6,21.4,19.5,6.5,2019,0,1,GSW


All champions and the outcomes have been incorporated correctly. This dataset will be saved and used for analysis in EDA and modeling

In [189]:
#save to file
df_team.to_csv('teams.csv')